In [1]:
!pip install -r req.txt

In [38]:
!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.5 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0


In [39]:
import torch 

print(torch.mps.is_available())

True


In [40]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
hugging_face_token = os.getenv("hf_token")
login(token=hugging_face_token)

In [41]:
import pandas as pd

label_map = {
    0: "world",
    1: "sports",
    2: "business",
    3: "sci/tech"
}
imbalanced_data = pd.read_parquet("Data/ag_news_train_imbalanced.parquet")
balanced_data = pd.read_parquet("Data/ag_news_train_balanced.parquet")
imbalanced_data_5_to_1 = pd.read_parquet("Data/ag_news_train_imbalanced_5_to_1_ratio.parquet")

balanced_data["label"] = balanced_data["label"].map(label_map)
imbalanced_data["label"] = imbalanced_data["label"].map(label_map)
imbalanced_data_5_to_1["label"] = imbalanced_data_5_to_1["label"].map(label_map)

# Shuffle the dataset
imbalanced_data = imbalanced_data.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
imbalanced_data_5_to_1 = imbalanced_data_5_to_1.sample(frac=1).reset_index(drop=True)

# Testing
balanced_data.shape



(2000, 2)

In [42]:
def build_prompt(text):
    """
    Function to construct an instruction for the LLM

    Args:
        text (str): The text of the data

    Returns:
        prompt (str): The constructed prompt for the LLM
    """
    prompt = (
        f"You are a powerful, precise, and helpful assistant that classifies text into well-defined categories."
        f" Your task is to carefully analyze the meaning, tone, and intent of the given text, then select the most appropriate category."
        f" Choose only from the following categories: {', '.join(list(label_map.values()))}."
        f" Respond with exactly one word: the single best category."
        f" Do not explain your choice, provide reasoning, or output anything else."
        f" Learn from these examples to understand context and edge cases: "

    )

    few_shots_example = [
        {"text": "Macedonian Prime Minister Kostov Resigns Parliament will officially announce Prime Minister Hari Kostov #39;s resignation during a meeting scheduled for Thursday, launching a ten-day period during which President Branko Crvenkovski should give the mandate for forming a new government.", 'label': 'business'},
        {'text': 'Trinidad climbs off canvas to keep title options open Felix Trinidad returned to the ring after more than two years to score a thrilling eighth-round stoppage of Ricardo Mayorga at New York #39;s Madison Square Garden in a non- title bout being described as one of the fights of the year.', 'label': 'sports'},
        {'text': 'Hungary #39;s ruling Socialist Party dumps PM BUDAPEST, Aug. 19 (Xinhuanet) -- Hungary #39;s ruling Socialist Party said Thursday that it accepted Prime Minister Peter Medgyessy #39;s resignation and has named a candidate for the post.', 'label': 'world'}
    ]

    prompt += "\n\n"
    for ex in few_shots_example:
        prompt += f"Review: \"{ex['text']}\"\nCategory: {ex['label']}\n\n"
    prompt += f"Review: \"{text}\"\nCategory:" #Leave Category here blank since we want the LLM to generate text
    return prompt


# Testing function
print(build_prompt("Astros Rally Past the Giants With one swing of the bat, Lance Berkman revived the Houston Astros' playoff hopes - and gave the Los Angeles Dodgers a much-needed reprieve. Berkman hit a three-run homer off Dustin Hermanson, highlighting a five-run ninth inning that sent Houston to a 7-3 win over San Francisco on Thursday night..."))

You are a powerful, precise, and helpful assistant that classifies text into well-defined categories. Your task is to carefully analyze the meaning, tone, and intent of the given text, then select the most appropriate category. Choose only from the following categories: world, sports, business, sci/tech. Respond with exactly one word: the single best category. Do not explain your choice, provide reasoning, or output anything else. Learn from these examples to understand context and edge cases: 

Review: "Macedonian Prime Minister Kostov Resigns Parliament will officially announce Prime Minister Hari Kostov #39;s resignation during a meeting scheduled for Thursday, launching a ten-day period during which President Branko Crvenkovski should give the mandate for forming a new government."
Category: business

Review: "Trinidad climbs off canvas to keep title options open Felix Trinidad returned to the ring after more than two years to score a thrilling eighth-round stoppage of Ricardo Mayo

In [ ]:
# Load the model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(
    model,
    torch_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Move model to mps (If using Mac)
# if torch.backends.mps.is_available():
#     model.to('mps')

# model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [47]:

from tqdm.auto import tqdm
from transformers import pipeline, logging
import time


# Load model
model = "microsoft/phi-2" #Could be changed later for more evals

# CREATE A FUNCTION TO RUN CLASSFICATION
def classify(model, tokenizer, df, batch_size):
    """
    Function to run classification on 3 datasets, using batched prediction

    Args:
        model (str): name of the model
        tokenizer
        df (pd.DataFrame): the pandas dataframe
        batch_size (int): batch size per run

    Returns:
        None
    """
    # Initiate a pipeline for each dataset
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    logging.set_verbosity_error()

    # Generate prompts for all rows
    prompts = [build_prompt(text) for text in df["text"]]
    
    # Run the pipeline for each row
    pred_arr = []
    start_time = time()
    
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size] #slices a sublist of prompts
        results = pipe(batch, max_new_tokens=3, do_sample=False, pad_token_id=tokenizer.eos_token_id)
        for prompt, res in zip(batch, results):
            pred = res[0]['generated_text'][len(prompt):].strip().split()[0]
            pred_arr.append(pred)
    end_time = time()
    # Print total running time
    print(f"Total running time is {end_time - start_time} seconds")
    df["llm_prediction"] = pred_arr
    print("Predictions have been added to the dataframe")



In [49]:
# Run with 3 datasets
bs = 4

classify(
    model=model,
    tokenizer=tokenizer,
    df=balanced_data.iloc[:10],
    batch_size=bs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0


RuntimeError: MPS backend out of memory (MPS allocated: 18.05 GB, other allocations: 464.00 KB, max allowed: 18.13 GB). Tried to allocate 500.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).